In [6]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV

from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


In [52]:
df = pd.read_csv('data_spam.csv')
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


# Data Cleansing

In [53]:
df.info() # non NaN

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   object
 1   Message   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [50]:
df = pd.get_dummies(df, columns=['Category'])

KeyError: "None of [Index(['Category'], dtype='object')] are in the [columns]"

In [54]:
df.head()


,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [65]:
# replace Category 
dict_trans = {'ham' : 0, 'spam': 1}
df = df.replace({"Category": dict_trans})

# Data Preparation

In [66]:
# train-/test-split

labels = df['Category'].values
messages = df['Message'].values
messages_train, messages_test, y_train, y_test = train_test_split(messages, labels, test_size=0.3, random_state=42)

array([0, 0, 1, ..., 0, 0, 0])

In [69]:
# Vectorisation
vectorizer = CountVectorizer()
vectorizer.fit(sentences_train)

X_train = vectorizer.transform(messages_train)
X_test  = vectorizer.transform(messages_test)

In [70]:
X_train

<3900x1 sparse matrix of type '<class 'numpy.int64'>'
	with 65 stored elements in Compressed Sparse Row format>

# Build First Model

In [72]:

input_dim = X_train.shape[1]  # Number of features

In [73]:


model = Sequential()
model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [74]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 10)                20        
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 11        
Total params: 31
Trainable params: 31
Non-trainable params: 0
_________________________________________________________________


In [76]:
history = model.fit(X_train, y_train,
                    epochs=100,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=10)

Train on 3900 samples, validate on 1672 samples
Epoch 1/100
3900/3900 [==============================] - 1s 154us/step - loss: 0.3917 - accuracy: 0.8667 - val_loss: 0.3913 - val_accuracy: 0.8660
Epoch 2/100
3900/3900 [==============================] - 1s 145us/step - loss: 0.3916 - accuracy: 0.8667 - val_loss: 0.3913 - val_accuracy: 0.8660
Epoch 3/100
3900/3900 [==============================] - 1s 145us/step - loss: 0.3919 - accuracy: 0.8667 - val_loss: 0.3913 - val_accuracy: 0.8660
Epoch 4/100
3900/3900 [==============================] - 1s 146us/step - loss: 0.3917 - accuracy: 0.8667 - val_loss: 0.3913 - val_accuracy: 0.8660
Epoch 5/100
3900/3900 [==============================] - 1s 148us/step - loss: 0.3917 - accuracy: 0.8667 - val_loss: 0.3913 - val_accuracy: 0.8660
Epoch 6/100
3900/3900 [==============================] - 1s 145us/step - loss: 0.3917 - accuracy: 0.8667 - val_loss: 0.3913 - val_accuracy: 0.8660
Epoch 7/100
3900/3900 [==============================] - 1s 145us/step

3900/3900 [==============================] - 1s 146us/step - loss: 0.3916 - accuracy: 0.8667 - val_loss: 0.3912 - val_accuracy: 0.8660
Epoch 57/100
3900/3900 [==============================] - 1s 146us/step - loss: 0.3916 - accuracy: 0.8667 - val_loss: 0.3913 - val_accuracy: 0.8660
Epoch 58/100
3900/3900 [==============================] - 1s 146us/step - loss: 0.3915 - accuracy: 0.8667 - val_loss: 0.3914 - val_accuracy: 0.8660
Epoch 59/100
3900/3900 [==============================] - 1s 150us/step - loss: 0.3917 - accuracy: 0.8667 - val_loss: 0.3913 - val_accuracy: 0.8660
Epoch 60/100
3900/3900 [==============================] - 1s 149us/step - loss: 0.3916 - accuracy: 0.8667 - val_loss: 0.3913 - val_accuracy: 0.8660
Epoch 61/100
3900/3900 [==============================] - 1s 168us/step - loss: 0.3916 - accuracy: 0.8667 - val_loss: 0.3913 - val_accuracy: 0.8660
Epoch 62/100
3900/3900 [==============================] - 1s 159us/step - loss: 0.3917 - accuracy: 0.8667 - val_loss: 0.3913 

In [77]:
# model performance
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 0.8667
Testing Accuracy:  0.8660
